# Trie-with ord function

In [1]:
class Node:
    def __init__(self):
        self.child = [None] * 26
        self.end = False
        self.count = 0  # To track the count of words passing through this node

class Tree:
    def __init__(self):
        self.root = Node()

    def insert(self, key):
        current = self.root
        for i in key:
            index = ord(i) - ord('a')
            if not current.child[index]:
                current.child[index] = Node()
            current = current.child[index]
            current.count += 1  # Increment count of words passing through
        current.end = True
        

    def display(self):
        def dsp(current, word):
            if current.end:
                print(word)
            for i in range(26):
                if current.child[i]:
                    dsp(current.child[i], word + chr(i + ord('a')))
        dsp(self.root, '')

    def search(self, key):
        current = self.root
        for i in key:
            index = ord(i) - ord('a')
            if not current.child[index]:
                return False
            current = current.child[index]
        return current.end

    def prefixsearch(self, key):
        current = self.root
        for i in key:
            index = ord(i) - ord('a')
            if not current.child[index]:
                return False
            current = current.child[index]
        return True

    def delete_key(self, key):
        def _delete(curr, key, depth):
            if curr is None:
                return False
            if depth == len(key):
                if curr.end:
                    curr.end = False
                    return not any(curr.child)  # Delete node if it has no children
                return False
            index = ord(key[depth]) - ord('a')
            should_delete_current_node = _delete(curr.child[index], key, depth + 1)
            if should_delete_current_node:
                curr.child[index] = None
                return not curr.end and not any(curr.child)
            return False

        return _delete(self.root, key, 0)

    def autocomplete(self, prefix):
        current = self.root
        for char in prefix:
            index = ord(char) - ord('a')
            if not current.child[index]:
                return []
            current = current.child[index]

        result = []
        self._collect_words(current, prefix, result)
        return result

    def _collect_words(self, node, word, result):
        if node.end:
            result.append(word)
        for i in range(26):
            if node.child[i]:
                self._collect_words(node.child[i], word + chr(i + ord('a')), result)

    def longest_prefix(self):
        current = self.root
        prefix = ''
        while current and sum(child is not None for child in current.child) == 1 and not current.end:
            for i in range(26):
                if current.child[i]:
                    prefix += chr(i + ord('a'))
                    current = current.child[i]
                    break
        return prefix

    def count_words_with_prefix(self, prefix):
        current = self.root
        for char in prefix:
            index = ord(char) - ord('a')
            if not current.child[index]:
                return 0
            current = current.child[index]
        return current.count

    def find_unique_prefix(self, key):
        current = self.root
        prefix = ''
        for char in key:
            index = ord(char) - ord('a')
            if current.child[index].count == 1:
                prefix += char
                return prefix
            prefix += char
            current = current.child[index]
        return prefix

# Testing the Tree with new functions
ob = Tree()
ob.insert('abcd')
ob.insert('abcde')
ob.insert('abc')
ob.insert('how')
ob.insert('home')

ob.display()
print("Longest Prefix:", ob.longest_prefix())  # Should return 'abc' because 'abcd', 'abcde', 'abc' share it
print("Count Words with Prefix 'ho':", ob.count_words_with_prefix('ho'))  # Should return 2 ('how', 'home')
print("Unique Prefix of 'home':", ob.find_unique_prefix('home'))  # Should return 'hom'

abc
abcd
abcde
home
how
Longest Prefix: 
Count Words with Prefix 'ho': 2
Unique Prefix of 'home': hom


# Trie-in normal indexing way

In [9]:
class trinode:
    def __init__(self):
        self.children = {}
        self.end_word = False

class trie:
    def __init__(self):
        self.root = trinode()

    def insert(self, word):
        node = self.root
        for ch in word:
            if ch not in node.children:
                node.children[ch] = trinode()
            node = node.children[ch]
        node.end_word = True

    def search(self, word):
        node = self.root
        for ch in word:
            if ch not in node.children:
                return False
            node = node.children[ch]
        return node.end_word

    def start_with(self, prefix):
        node = self.root
        for ch in prefix:
            if ch not in node.children:
                return False
            node = node.children[ch]
        return True

    def display(self, node=None, word=""):
        if node is None:
            node = self.root
        if node.end_word:
            print(word)
        for ch, child in node.children.items():
            self.display(child, word + ch)

    def autocomplete(self, prefix):
        node = self.root
        for ch in prefix:
            if ch not in node.children:
                return []
            node = node.children[ch]
        words = []

        def dfs(n, path):
            if n.end_word:
                words.append(path)
            for ch, child in n.children.items():
                dfs(child, path + ch)

        dfs(node, prefix)
        return words

    def delete(self, word):
        def _delete(node, word, depth=0):
            if depth == len(word):
                if not node.end_word:
                    return False
                node.end_word = False
                return len(node.children) == 0

            ch = word[depth]
            if ch not in node.children:
                return False

            should_delete = _delete(node.children[ch], word, depth + 1)

            if should_delete:
                del node.children[ch]
                return len(node.children) == 0 and not node.end_word

            return False

        return _delete(self.root, word)

    def longest_prefix(self):
        node = self.root
        prefix = ""
        while node and len(node.children) == 1 and not node.end_word:
            ch = next(iter(node.children))
            prefix += ch
            node = node.children[ch]
        return prefix

    def unique_word_count(self):
        count = 0
        def dfs(node):
            nonlocal count
            if node.end_word:
                count += 1
            for child in node.children.values():
                dfs(child)
        dfs(self.root)
        return count

    def count_words_with_prefix(self, prefix):
        node = self.root
        for ch in prefix:
            if ch not in node.children:
                return 0

            node = node.children[ch]

        count = 0
        def dfs(n):
            nonlocal count
            if n.end_word:
                count += 1
            for child in n.children.values():
                dfs(child)
        dfs(node)
        return count
ob = trie()
ob.insert("sheikh")
ob.insert("shaanu")
ob.insert("abhin")

print("📜 Display All Words:")
ob.display()

print("\n🔍 Autocomplete for 'a':", ob.autocomplete("a"))
print("❌ Delete 'shaanu':", ob.delete("shaanu"))
print("\n📜 Display After Deletion:")
ob.display()

print("\n🔗 Longest Common Prefix:", ob.longest_prefix())
print("🔢 Total Unique Words:", ob.unique_word_count())
print("📦 Words Starting with 'sheikh':", ob.count_words_with_prefix("sheikh"))



📜 Display All Words:
sheikh
shaanu
abhin

🔍 Autocomplete for 'a': ['abhin']
❌ Delete 'shaanu': False

📜 Display After Deletion:
sheikh
abhin

🔗 Longest Common Prefix: 
🔢 Total Unique Words: 2
📦 Words Starting with 'sheikh': 1
